This is a recommender system built using K nearest neighbors

In [377]:
#!/usr/bin/env python
# coding: utf-8

# general purpose libraries
import platform
import os
import dateutil
import itertools

# database connection libraries
import sqlite3
from sqlite3 import Error

# mathematics libraries
import pandas as pd
import numpy as np

# ML libraries
from sklearn.neighbors import KDTree                    # Nearest neighbors
from sklearn.preprocessing import StandardScaler        # scales all values between a range 
from sklearn.model_selection import train_test_split    # cross-validation
from tpot import TPOTRegressor                          # genetic algorithm implementation
from sklearn.externals import joblib                    # model loading
import pickle                                           # model saving

# data visualisation libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

this Function connects with the SQL DB

In [378]:
def create_connection(db_file):
        """ create a database connection to the SQLite database
            specified by the db_file
        :param db_file: database file
        :return: Connection object or None
        """
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return None

Connecting all the DBs

In [379]:
Sql_connection1 = create_connection('/opt/apps/scripts/jaarvis_demand_supply/evo.db')
Sql_connection = create_connection('/opt/apps/scripts/jaarvis_demand_supply/zone_travel.db')

Here we are checking what tables are there in our DB

In [402]:
cursor = Sql_connection.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('zone_travel_detail',)]


In [465]:
technician_db = pd.read_sql_query("select * from zone_technician;", Sql_connection1)
technician_db = technician_db.fillna(-9999)

In [466]:
avg_trip_cost = pd.read_sql_query("select * from zone_trip_avg;", Sql_connection1)


In [467]:
zone_detail_df = pd.read_sql_query("select * from zone_travel_detail;", Sql_connection)
# zone_detail_df.head()

Data Preprocessing

In [468]:
temp = zone_detail_df.loc[zone_detail_df['to_zone'].isin(list(avg_trip_cost.zone.unique()))]
temp = temp.loc[temp['from_zone'].isin(list(avg_trip_cost.zone.unique()))]

zone_detail_df = temp

In [469]:
avg_trip_cost.total_trips = avg_trip_cost.total_trips.astype(int)

In [470]:
avg_trip_cost = avg_trip_cost.sort_values(by = 'zone')

In [471]:
# bringing all DFs in same format so that they can be joined to pass to the ML model
cost_diff=pd.DataFrame(list(itertools.product(avg_trip_cost.zone,avg_trip_cost.zone)),columns=['from_zone','to_zone'])
dic = dict(zip(avg_trip_cost.zone,avg_trip_cost.avg_cost))
cost_diff['cost_diff']=cost_diff.from_zone.map(dic)-cost_diff.to_zone.map(dic)
cost_diff.head()

cost_diff = cost_diff.fillna(-9999)

In [472]:
# Missing values handling
technician_db = technician_db.loc[technician_db['zone'].isin(list(avg_trip_cost.zone.unique()))]

In [473]:
technician_db.avg.loc[technician_db['avg'] < 3 ] = -15
technician_db.avg.loc[technician_db['avg'] > 10] = -1
technician_db.avg.loc[technician_db['avg'] > 3] = 10
technician_db.avg.loc[technician_db['avg'] == -1] = 0

technician_db = technician_db.fillna(-9999)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [474]:
# missing_zones = pd.DataFrame({'id':[],'avg':[],'zone':[],'from_time':[],'to_time':[]})
# missing_zones.zone= avg_trip_cost.zone[~avg_trip_cost.zone.isin(technician_db.zone)]

In [475]:
# temp_df = pd.concat([technician_db,missing_zones],ignore_index=True)

In [476]:
technician_db = technician_db.drop_duplicates('zone', keep='last')

In [477]:
import itertools
tech_diff=pd.DataFrame(list(itertools.product(technician_db.zone,technician_db.zone)),columns=['from_zone','to_zone'])
dic = dict(zip(technician_db.zone,technician_db.avg))
tech_diff['tech_diff']=tech_diff.from_zone.map(dic)

tech_diff = tech_diff.fillna(-9999)



In [487]:
# final DF after data preprocessing & cleaning & joining
supply_df = pd.merge(zone_detail_df, tech_diff,  how='left', left_on=['from_zone','to_zone'], right_on = ['from_zone','to_zone'])
supply_df = pd.merge(supply_df,cost_diff ,  how='left', left_on=['from_zone','to_zone'], right_on = ['from_zone','to_zone'])
supply_df = supply_df.fillna(-9999)

In [488]:
supply_df['vector'] = list(map(list,(zip(supply_df.kilometers,supply_df.minutes,supply_df.cost_diff,supply_df.tech_diff))))

In [489]:
supply_pivot_df = pd.pivot_table(supply_df,values=['vector'],index=['from_zone'],columns=['to_zone'],aggfunc='first')

In [490]:
supply_pivot_df = supply_pivot_df.vector

In [491]:
supply_pivot_df = supply_pivot_df.sort_index()
supply_pivot_df = supply_pivot_df[supply_pivot_df.columns.sort_values()]


In [492]:
supply_pivot_df=supply_pivot_df.applymap(lambda x: [-9999,-9999,-9999,-9999] if x is np.nan else x)
supply_pivot_df = supply_pivot_df[supply_pivot_df.index.isin(supply_pivot_df.columns)]
supply_pivot_df = supply_pivot_df.loc[:, supply_pivot_df.columns.isin(supply_pivot_df.index)]

In [540]:
for i in range(supply_pivot_df.shape[0]):
    supply_pivot_df.iloc[i,i]=[0,0,0,0]

weighted_score = time*x1+distance*x2+average_trip_cost*x3+no_of_tech*x4+cost_of_relocating*x5+hot_zone*x6+supply gap*x7

Garstin_Bastion = 

In [577]:
# def nearest_neighbors():
from sklearn.neighbors import KDTree
neighbor_list=[]
for x in range(0,supply_pivot_df.shape[0]):
    df  = [np.array(j) for i,j in supply_pivot_df.iloc[x,:].iteritems()]
    kdt = KDTree(df, leaf_size=30, metric='euclidean')
    zone_neighbors = kdt.query(df, k=5, return_distance=False)[x]
    neighbor_list.append([supply_pivot_df.columns[i] for i in zone_neighbors])

In [341]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import numpy as np
# # X, Y, Z, U, V, W = zip(*df)
# # fig = plt.figure()
# # ax = fig.add_subplot(111, projection='3d')
# # ax.quiver(X, Y, Z, U, V, W)

# # ax.set_xlim([-2, 2])
# # ax.set_ylim([-2, 2])
# # ax.set_zlim([-2, 2])
# # ax.legend()
# # plt.show()

# avg_trip_cost = avg_trip_cost[2:]
# # np.where(avg_trip_cost.zone =='V5Z 2L5')
# # df = avg_trip_cost.set_index('zone')
# # mux = pd.MultiIndex.from_product([avg_trip_cost.zone, avg_trip_cost.zone], names=['name1','name2'])

# # df = (avg_trip_cost['avg_cost'].reindex(mux, level=0)
# #         .sub(df.reindex(mux, level=1)['avg_cost'])
# #         .rename('time_diff')
# #         .reset_index())
# supply_df = supply_df[numpy.isfinite(supply_df['cost_diff'])]
# supply_df = supply_df[numpy.isfinite(supply_df['kilometers'])]
# list(set(supply_df.to_zone) - set(supply_df.from_zone))
# # zone_detail_df.to_zone.isin()
len(set(avg_trip_cost.zone.unique()).difference((set(zone_detail_df.to_zone.unique()))))


30

In [286]:
# demand = pd.read_json('http://139.59.93.34:5000/demand')
# live_cars = pd.read_json('live_cars.json')
# demand = demand.T
# live_cars = live_cars.T
# # demand = demand.fillna(0)
# # live_cars = live_cars.fillna(0)

# def type_conversion(x):
#     if type(x)== list:
#         return tuple(x)
#     else:
#         print(x)
    
# live_cars.lat_long = live_cars.lat_long.apply(lambda x: type_conversion(x))
# demand.lat_long = demand.lat_long.apply(lambda x: type_conversion(x))

# result= pd.merge(live_cars,demand, how='outer',on='lat_long')



nan


In [578]:
neighbor_list

[['14640', 'V5V 3E2', 'V5R 5K6', 'V5W 1G5', 'V5M 4C1'],
 ['V1M 2R6', 'V1M 3G1', 'V3C 2S9', 'V3L 3P2', 'V3L 3X6'],
 ['V1M 3G1', 'V3C 2S9', 'V3L 3P2', 'V3L 3X6', 'V3L 3V9'],
 ['V2J 3A9', 'V3L 3P2', 'V3L 3X6', 'V3K 7B3', 'V3L 3V9'],
 ['V3C 2S9', 'V3L 3P2', 'V3L 3X6', 'V3L 3V9', 'V3L 3H7'],
 ['V3J 3Z9', 'V3K 7B3', 'V3L 3X6', 'V5A 4X6', 'V3L 3X1'],
 ['V3K 7B3', 'V3L 3P2', 'V3L 3X6', 'V3L 4C9', 'V3L 3X1'],
 ['V3L 0A5', 'V3L 2T3', 'V3L 4C9', 'V3N 3N4', 'V3L 0H4'],
 ['V3L 0C9', 'V3L 3A9', 'V3L 3B2', 'V3M 3K3', 'V3L 1K3'],
 ['V3L 0E3', 'V3L 3V9', 'V3L 3H7', 'V3L 3X1', 'V3L 0H4'],
 ['V3L 0E7', 'V3L 4G6', 'V3L 0H4', 'V3L 2G1', 'V3L 1A7'],
 ['V3L 0H4', 'V3L 3H7', 'V3L 4G6', 'V3L 3V9', 'V3L 4C9'],
 ['V3L 1A7', 'V3L 1A9', 'V3L 1B9', 'V3L 1G2', 'V3L 2K1'],
 ['V3L 1A9', 'V3L 1B9', 'V3L 1G2', 'V3L 2K1', 'V3L 2G1'],
 ['V3L 1B9', 'V3L 1G2', 'V3L 2K1', 'V3L 2G1', 'V3L 1K3'],
 ['V3L 1G2', 'V3L 2K1', 'V3M 3K3', 'V3L 3A9', 'V3L 2G1'],
 ['V3L 1K3', 'V3M 3K3', 'V3L 3A9', 'V3L 2G1', 'V3L 3B2'],
 ['V3L 2G1', 'V3

In [573]:
for i in kdt.query(df, k=5, return_distance=False)[x]:
    print(supply_pivot_df.columns[i])

V7R 4T8
V6E 1V1
V6E 4R5
V6E 4H6
V6E 1L6


In [581]:
live_cars_result()

connection to DB file successful!.....
data loaded successfully!....
                        Lat_Lon
0     [49.2784729, -122.909424]
1       [49.2777863, -122.9121]
2       [49.27774, -122.912086]
3     [49.2774849, -122.912193]
4     [49.2304573, -122.886292]
5     [49.2304649, -122.886337]
6       [49.23193, -122.887505]
7     [49.2318573, -122.887505]
8      [49.279808, -122.923683]
9        [49.27993, -122.92382]
10    [49.2300644, -122.886368]
11      [49.23061, -122.890114]
12    [49.2282677, -122.888954]
13    [49.2294769, -122.890251]
14      [49.22822, -122.891472]
15     [49.230217, -122.894249]
16    [49.2284431, -122.893921]
17      [49.22935, -122.894722]
18    [49.2284164, -122.894081]
19      [49.22941, -122.894882]
20      [49.23089, -122.896111]
21      [49.23005, -122.896324]
22      [49.22646, -122.894844]
23    [49.2246437, -122.894051]
24    [49.2310867, -122.900383]
25      [49.22462, -122.897362]
26     [49.231678, -122.902786]
27     [49.222023, -122.895645]
28 